In [18]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from random import random, seed
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline
from sklearn.utils import resample
from joblib import Parallel, delayed
from functions import *
plt.rcParams.update({'text.usetex': True, 'font.size': 16, 'font.family': 'serif', 'font.serif': 'Computer Modern Sans Serif', 'font.weight': 100, 'mathtext.fontset': 'cm', 'xtick.labelsize': 14, 'ytick.labelsize': 14})

seed = 43
np.random.seed(seed)

with open("../data/darkmatter_1288.txt", "r") as infile:
    params = infile.readline().split(",")
    # The box unit, density unit, dimensionless Hubble constant, x and y are the same for both redshifts
    box_unit, rho_unit, h, a_1288 = params[0].split(": ")[1], params[1].split(": ")[1], float(params[2].split(": ")[1]), float(params[3].split(": ")[1]) 
    y = np.array(infile.readline().split(), dtype = float)
    x = []; rho_1288 = []
    for line in infile:
        vals = line.split()
        x.append(float(vals[-1]))
        rho_1288.append(np.array(vals[:-1], dtype = float))
x = np.array(x); rho_1288 = np.array(rho_1288)

# The box unit, density unit, dimensionless Hubble constant, x and y are the same for both redshifts, so we only store the values of a and ρ this time

with open("../data/darkmatter_0.txt", "r") as infile:
    params = infile.readline().split(",")
    a_0 = float(params[3].split(": ")[1]) 
    infile.readline()
    rho_0 = []
    for line in infile:
        vals = line.split()
        rho_0.append(np.array(vals[:-1], dtype = float))
rho_0 = np.array(rho_0)

x.reshape(-1, 1); y.reshape(-1, 1) 
z_1288 = np.log(rho_1288).reshape(-1, 1) 
z_0 = np.log(rho_0).reshape(-1, 1) 

In [19]:
lambdas = np.logspace(-7, 0, 5)
degrees = range(1, 7)
MSE_Lasso_list = []
R2_Lasso_list = []
z_list = [z_1288, z_0]

In [20]:
for z in z_list:
    MSE_Lasso_list.append(np.zeros((len(degrees), len(lambdas))))
    R2_Lasso_list.append(np.zeros((len(degrees), len(lambdas))))
    for i in range(len(degrees)):
        print(f"Degree {degrees[i]}")
        for j in range(len(lambdas)):
            MSE_Lasso, R2_Lasso = Lasso(x, y, z, degrees[i], lambdas[j], seed = seed)
            MSE_Lasso_list[-1][i, j] = MSE_Lasso
            R2_Lasso_list[-1][i, j] = R2_Lasso

Degree 1
Degree 2
Degree 3
Degree 4
Degree 5
Degree 6
Degree 1
Degree 2
Degree 3
Degree 4
Degree 5
Degree 6


In [21]:
result_list = []

for i, z in enumerate(z_list):
    result_list.append(Parallel(n_jobs = -1)
    (delayed(Lasso)(x, y, z, degree, lambdas[j], seed = seed) 
        for degree in degrees
            for j in range(len(lambdas))))